Before we begin, let's execute the cell below to display information about the CUDA driver and GPUs running on the server by running the `nvidia-smi` command. To do this, execute the cell block below by giving it focus (clicking on it with your mouse), and hitting Ctrl-Enter, or pressing the play button in the toolbar above. If all goes well, you should see some output returned below the grey cell.

In [ ]:
!nvidia-smi

## Learning objectives
The **goal** of this lab is to:

- Dig deeper into kernels by analyzing it with Nsight Compute

In the previous section, we looked at some of the ways to optimize the parallel [RDF](../serial/rdf_overview.ipynb) application using OpenMP offloading. Moreover, we used NVIDIA Nsight Systems to get a system-wide performance analysis. Now, let's dig deeper and profile the kernel with the Nsight Compute profiler to get detailed performance metrics. Note: You will get a better understanding of the GPU architecture in the CUDA notebooks.


To do this, let's use the [solution](../../source_code/openmp/SOLUTION/rdf_offload_split.cpp) as a reference to get a similar report from Nsight Compute. Run the application, and profile it with the Nsight Systems first.

Now, let's compile, and profile it with Nsight Systems first.

In [ ]:
#compile for Tesla GPU
!cd ../../source_code/openmp && nvc++ -mp=gpu -Minfo=mp -o rdf SOLUTION/rdf_offload_split.cpp 

In [ ]:
#profile and see output of nvptx
!cd ../../source_code/openmp && nsys profile -t nvtx,cuda --stats=true --force-overwrite true -o rdf_offload_split ./rdf

Let's checkout the profiler's report. [Download the profiler output](../../source_code/openmp/rdf_offload_split.qdrep) and open it via the GUI. Now, right click on the kernel and click on "Analyze the Selected Kernel with NVIDIA Nsight Compute" (see below screenshot). 

<img src="../images/compute_analyz.png">

Then, make sure to tick the radio button next to "Display the command line to user NVIDIA Nsight Compute CLI". 

<img src="../images/compute_command_line.png" width="50%" height="50%">

Then, you simply copy the command, run it and analyze the selected kernel. 

To profile the selected kernel, run the below cell (by adding `--set full` we make sure to capture all the sections in Nsight Compute profiler):

In [ ]:
#profile with nsight compute
!cd ../../source_code/openmp && ncu --set full --launch-skip 0 --launch-count 1 -k pair_gpu -o rdf_offload_split ./rdf

Let's checkout Nsight Compute report. [Download the profiler output](../../source_code/openmp/rdf_offload_split.ncu-rep) and open it via the GUI. Have a look at the example expected profiler report below:

<img src="../images/openmp_offload_split_cmp.png">

When compared to the base version using the baseline feature on the Nsight Compute (base version is the very first parallel version using OpenMP), we achieved 72% improvement in SM utilization and 123% utilization in memory. 

### `num_teams(n)` and `thread_limit`

When using the `distribute parallel for` construct, the *for loop* is distributed across all [threads](../GPU_Architecture_Terminologies.ipynb) for all teams of the current teams region. For example, if there are 10 teams, and each team consists of 256 threads, the loop will be distributed across 2560 threads. You can explicitly specify the number of threads to be created in the team using the `thread_limit(m)`. Moreover, you can specify the maximum number of teams created by using `num_teams(n)` (the actual number of teams may be smalled than this number). You can add these attributes to the `teams` construct. Please note that you cannot add `num_teams` to the `parallel` construct clause. For more information, please read the guide at https://docs.nvidia.com/hpc-sdk/compilers/hpc-compilers-user-guide/#openmp-subset .


```cpp
#pragma omp target teams distribute num_teams(65535)
for (int i = 0; i < N; i++ )
{
    #pragma parallel for thread_limit(128)
    for (int j=0;j< M;j++)
        < loop code >
} 
```
In the previous section, the grid size was too small to keep the hardware busy. Take a look at the *Launch Statistics*, the grid size is 80. 

<img src="../images/openmp_offload_split_grid.png">

Now, lets start modifying the code again and add `thread_limit(n)` to the `teams` construct. From the top menu, click on *File*, and *Open* `rdf.cpp` and `dcdread.h` from the current directory at `C/source_code/openmp` directory. Remember to **SAVE** your code after changes, before running below cells.

**NOTE:** Try increasing the grid size to 65525 (*hint:* use `num_teams(65535)`). Once done, compile and profile the code and compare it with the previous version. Are there any differences? What does the profiler show?

In [ ]:
#compile for Tesla GPU
!cd ../../source_code/openmp && nvc++ -mp=gpu -Minfo=mp -o rdf rdf.cpp 

Now, validate the output by running the executable, and then **Profile** your code with Nsight Systems command line `nsys`.

In [ ]:
#Run on Nvidia GPU and check the output
!cd ../../source_code/openmp && ./rdf && cat Pair_entropy.dat

The output should be the following:

```
s2 value is -2.43191
s2bond value is -3.87014
```

In [ ]:
#profile and see output of nvptx
!cd ../../source_code/openmp && nsys profile -t nvtx,cuda --stats=true --force-overwrite true -o rdf_offload_split_num ./rdf

Let's checkout the profiler's report. [Download the profiler output](../../source_code/openmp/rdf_offload_split_num.qdrep) and open it via the GUI. Let's dig deeper and profile the `pair_gpu` kernel with Nsight Compute and compare it with the baseline.

In [ ]:
#profile with nsight compute
!cd ../../source_code/openmp && ncu --set full --launch-skip 0 --launch-count 1 -k pair_gpu -o rdf_offload_split_num ./rdf

Let's checkout the Nsight Compute report. [Download the profiler output](../../source_code/openmp/rdf_offload_split_num.ncu-rep) and open it via the GUI. 

<img src="../images/openmp_offload_split_cmp2.png">

Comparing to the previous version, we achieved 173% utilization in memory but 32% less SM utilization. Now, let's compare the two optimization approaches with the base version (using target offload). As seen in the roofline chart below, the application is now in the *memory bound* region which means either it is limited by memory bandwidth or latency.

<img src="../images/openmp_offload_roofline.png">


The kernel exhibits low compute throughput and memory bandwidth utilization relative to the peak performance of the device. Let's have a look at the *Warp State Statistics* section for potential reasons.

<img src="../images/openmp_warp_cmp.png">

Comparing the two optimizations, warp is stalled waiting for the L1 instruction queue for local and global (LG) memory operations to be not full. The instruction queue is full. 

 It is very clear that we did not improve anything through previous approaches. In the following sections, we look at the `collapse` clause to improve the performance.

Feel free to checkout the example [solution](../../source_code/openmp/SOLUTION/rdf_offload_offload_split_num.cpp) to help you understand better.

## Collapse clause

Specifying the `collapse(n)` clause takes the next `n` tightly-nested loops, folds them into one, and applies the OpenMP directives to the new loop. Collapsing loops means that two loops of trip counts N and M respectively will be automatically turned into a single loop with a trip count of N times M. By collapsing two or more parallel loops into a single loop the compiler has an increased amount of parallelism to use when mapping the code to the device. On highly parallel architectures, such as GPUs, this will give us more parallelism to distribute and better performance.

Try using the collapse clause and observe any performance difference. How much this optimization will speed-up the code will vary according to the application and the target accelerator, but it is not uncommon to see large speed-ups by using collapse on loop nests.

In the example below, we collapse the two loops before applying both teams and thread parallelism to both.

```cpp
#pragma omp target teams distribute parallel for collapse(2) 
for (int i = 0; i < N; i++ )
{
    for (int j=0;j< M;j++)
        < loop code >
} 
```

Now, lets start modifying the original code before splitting the `teams distribute` from `parallel for` and add the collapse clause. From the top menu, click on *File*, and *Open* `rdf.cpp` and `dcdread.h` from the current directory at `C/source_code/openmp` directory. Remember to **SAVE** your code after changes, before running below cells.

In [ ]:
#compile for Tesla GPU
!cd ../../source_code/openmp && nvc++ -mp=gpu -Minfo=mp -o rdf rdf.cpp 

Now, validate the output by running the executable, and then **Profile** your code with Nsight Systems command line `nsys`.

In [ ]:
#Run on Nvidia GPU and check the output
!cd ../../source_code/openmp && ./rdf && cat Pair_entropy.dat

The output should be the following:

```
s2 value is -2.43191
s2bond value is -3.87014
```

In [ ]:
#profile and see output of nvptx
!cd ../../source_code/openmp && nsys profile -t nvtx,cuda --stats=true --force-overwrite true -o rdf_collapse ./rdf

Let's checkout the profiler's report. [Download the profiler output](../../source_code/openmp/rdf_collapse.qdrep) and open it via the GUI. Have a look at the example expected profiler report below:

<img src="../images/openmp_gpu_collapse.png">

Compare the execution time for the `Pair_Calculation` from the NVTX row (annotated in Red rectangle in the example screenshot) with the previous section. It is clear the using collapse clause improved the performance by extracting more parallelism. Please note when you compare the two methods explored here, one may give better results than the other.

Let's dig deeper and profile the kernel with Nsight Compute and compare it with the base version.

In [ ]:
#profile with nsight compute
!cd ../../source_code/openmp && ncu --set full --launch-skip 0 --launch-count 1 -k pair_gpu -o rdf_collapse ./rdf

Let's checkout the Nsight Compute report. [Download the profiler output](../../source_code/openmp/rdf_collapse.ncu-rep) and open it via the GUI. Have a look at the example expected profiler report below (yellow is the base version, blue is the current):

<img src="../images/openmp_offload_collapse.png">

When compared to the base version (using baseline feature on Nsight Compute), we achieved 58% improvement in SM utilization and 25% utilization in memory. We can also see the improvement and better utilization in the roofline analysis (see example screenshot below).The ideal situation would be getting closer to the rooflines (up).

<img src="../images/openmp_collapse_baseline.png">

Dots with the blue outline is the optimized version using `collapse` clause and the ones with the yellow outline is the base version.

Lets checkout the *Occupancy* section. Occupancy is the ratio of the number of active warps per multiprocessor to the maximum number of possible active warps. As seen in the below screenshot, the theoretical occupancy is 65% and the achieved occupancy is 58%. The plot showing the impact of varying register count per thread indicates that by reducing the number of registers per thread, we can increase the warp occupancy. 

<img src="../images/openmp_offload_occupancy.png">

We can limit the register count at compile time by adding `-gpu=maxregcount:32` flag. Based on the plot, let reduce the number of registers per thread to 32 to see if it improves the overall performance. 

In [ ]:
#compile for Tesla GPU
!cd ../../source_code/openmp && nvc++ -mp=gpu -gpu=maxregcount:32 -Minfo=mp -o rdf SOLUTION/rdf_offload_collapse.cpp 

Now, validate the output by running the executable, and then **Profile** your code with Nsight Systems command line `nsys`.

In [ ]:
#Run on Nvidia GPU and check the output
!cd ../../source_code/openmp && ./rdf && cat Pair_entropy.dat

The output should be the following:

```
s2 value is -2.43191
s2bond value is -3.87014
```

In [ ]:
#profile and see output of nvptx
!cd ../../source_code/openmp && nsys profile -t nvtx,cuda --stats=true --force-overwrite true -o rdf_offload_collapse_regcount ./rdf

Let's checkout the profiler's report. [Download the profiler output](../../source_code/openmp/rdf_collapse_regcount.qdrep) and open it via the GUI. Let's dig deeper and profile the `pair_gpu` kernel with Nsight Compute and compare it with the baseline.

In [ ]:
#profile with nsight compute
!cd ../../source_code/openmp && ncu --set full --launch-skip 0 --launch-count 1 -k pair_gpu -o rdf_collapse_regcount ./rdf

Let's checkout the Nsight Compute report. [Download the profiler output](../../source_code/openmp/rdf_collapse_regcount.ncu-rep) and open it via the GUI. Have a look at the example expected profiler report below (pink is the previous version using `collapse` clause, blue is the current after limiting the register count):

<img src="../images/openmp_collapse_reg.png">

We achieved 143% utilization in memory but SM utilization reduced by 6%. The current version takes 17% longer to execute. Roofline chart shows the application is now in memory bound region comparing to before that was in compute bound region (dots with the pink outline is the base version using `collapse` clause and the ones with the blue outline is the current version).

<img src="../images/openmp_collapse_reg_roofline.png">

Lets checkout the *Occupancy* section. As seen in the below screenshot, the theoretical occupancy is 100% and the achieved occupancy is 95%. Higher occupancy does not always result in higher performance, however, low occupancy always reduces the ability to hide latencies, resulting in overall performance degradation.


<img src="../images/openmp_collapse_reg_occupancy.png">

Let's have a look at the memory chart in the *memory workload analysis* section. Links between local memory and L1/TEX cache show an increase in the number of requests generated due to local memory load/store instructions. These are the result of registers spilling to local memory that happened after reducing the number of registers per thread to increase the occupancy. Once can also see the registers spilling by using `-Mcuda=ptxinfo` option at the compile time. Below is an example output showing the number of bytes spills to local memory. To read more about PTX, checkout the guide at https://docs.nvidia.com/cuda/parallel-thread-execution/index.html#ptx-machine-model .

``
ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function 'nvkernel__Z8pair_gpuPKdS0_S0_Pjiidddi_F1L174_1' for 'sm_70'
ptxas info    : Function properties for nvkernel__Z8pair_gpuPKdS0_S0_Pjiidddi_F1L174_1
    40 bytes stack frame, 52 bytes spill stores, 60 bytes spill loads
ptxas info    : Used 32 registers, 448 bytes cmem[0]
``

It is very clear that reducing the number of registers, had impact on memory and it hurt the performance by increasing the instruction count as well as memory traffic.

<img src="../images/openmp_collapse_reg_memory.png">

Feel free to checkout the example [solution](../../source_code/openmp/SOLUTION/rdf_offload_collapse.cpp) to help you understand better.

## Post-Lab Summary

If you would like to download this lab for later viewing, it is recommend you go to your browsers File menu (not the Jupyter notebook file menu) and save the complete web page.  This will ensure the images are copied down as well. You can also execute the following cell block to create a zip-file of the files you've been working on, and download it with the link below.

In [ ]:
%%bash
cd ..
rm -f nways_files.zip
zip -r nways_files.zip *

**After** executing the above zip command, you should be able to download the zip file [here](../nways_files.zip). Let us now go back to parallelizing our code using other approaches.

**IMPORTANT**: Please click on **HOME** to go back to the main notebook for *N ways of GPU programming for MD* code.

-----

# <p style="text-align:center;border:3px; border-style:solid; border-color:#FF0000  ; padding: 1em"> <a href=../../../nways_MD_start.ipynb>HOME</a></p>

-----


# Links and Resources
[OpenMP Programming Model](https://computing.llnl.gov/tutorials/openMP/)

[OpenMP Target Directive](https://www.openmp.org/wp-content/uploads/openmp-examples-4.5.0.pdf)

[NVIDIA Nsight System](https://docs.nvidia.com/nsight-systems/)

[NVIDIA Nsight Compute](https://developer.nvidia.com/nsight-compute)

**NOTE**: To be able to see the Nsight System profiler output, please download Nsight System latest version from [here](https://developer.nvidia.com/nsight-systems).

Don't forget to check out additional [OpenACC Resources](https://www.openacc.org/resources) and join our [OpenACC Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

--- 

## Licensing 

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).